In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.authorities.personalName import PersonalName

In [2]:
obj = {
  "id": "n80002329",
  "label": "Machado de Assis",
  "date": "1839-1908",
  "birthDate": "1839-06-21",
  "birthPlace": "Rio de Janeiro (Brazil)",
  "deathDate": "1908-09-29",
  "fullerName": "Joaquim Maria Machado",
  "hasCloseExternalAuthority": ["http://id.worldcat.org/fast/48407", "http://www.wikidata.org/entity/Q311145"],
  "hasExactExternalAuthority": ["http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept"],
  "hasVariant": [
    {
      "value": "Machado de Assis, Joaquín María",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Mashado de Assiz, Zhoakin",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Assis, Machado de",
      "type": "PersonalName",
      "date": "1839-1908"
    }
  ],
}

request = PersonalName(**obj)
request

PersonalName(recordChangeDate='2023-04-06T22:39:01', id='n80002329', label='Machado de Assis', date='1839-1908', birthDate='1839-06-21', birthPlace='Rio de Janeiro (Brazil)', deathDate='1908-09-29', fullerName='Joaquim Maria Machado', hasCloseExternalAuthority=['http://id.worldcat.org/fast/48407', 'http://www.wikidata.org/entity/Q311145'], hasExactExternalAuthority=['http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept'], hasVariant=[Authority(value='Machado de Assis, Joaquín María', lang=None, date='1839-1908', type='PersonalName'), Authority(value='Mashado de Assiz, Zhoakin', lang=None, date='1839-1908', type='PersonalName'), Authority(value='Assis, Machado de', lang=None, date='1839-1908', type='PersonalName')], type='PersonalName')

In [7]:
[hasVariant] = request.hasVariant
hasVariant.date



'1839-1908'

In [4]:
def MakeVariant(hasVariant):
    
    variants = list()
    for i in hasVariant:
        variant = f"""[ a madsrdf:PersonalName,
                madsrdf:Variant ;
            madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                        madsrdf:elementValue "{i.value}" ] 
                         { '[ a madsrdf:DateNameElement ; madsrdf:elementValue "{}" ] ) ;'.format(i.date) if i.date else ') ;' }           
            { ' madsrdf:variantLabel "{}, {}" ]'.format(i.value, i.date) if i.date else 'madsrdf:variantLabel "{i.value}" ]' }"""
        variants.append(variant)
    return variants

variants = MakeVariant(request.hasVariant)


['[ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                        madsrdf:elementValue "Machado de Assis, Joaquín María" ] \n                         [ a madsrdf:DateNameElement ; madsrdf:elementValue "1839-1908" ] ) ;           \n             madsrdf:variantLabel "Machado de Assis, Joaquín María, 1839-1908" ]', '[ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                        madsrdf:elementValue "Mashado de Assiz, Zhoakin" ] \n                         [ a madsrdf:DateNameElement ; madsrdf:elementValue "1839-1908" ] ) ;           \n             madsrdf:variantLabel "Mashado de Assiz, Zhoakin, 1839-1908" ]', '[ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                        madsrdf:elementValue "Assis, Machado de" ] \n              

In [11]:
closeExternal = "madsrdf:hasCloseExternalAuthority {} ; \n"
exactExternal = "madsrdf:hasExactExternalAuthority {} ; \n"
hasVariant = "madsrdf:hasVariant  {}  ; \n"
birthPlace = """madsrdf:birthPlace [ a madsrdf:Geographic ;
            rdfs:label "{}" ] ;"""

if request.hasVariant:
        variants = MakeVariant(request.hasVariant)

template = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX name: <https://bibliokeia.com/authorities/name/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT DATA {{
GRAPH name:{request.id} {{
 name:{request.id} a madsrdf:Authority, madsrdf:PersonalName ;
        madsrdf:adminMetadata [ a ri:RecordInfo ; 
        ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
        ri:recordStatus "new"^^xsd:string
        ] ; 
        madsrdf:authoritativeLabel "{request.label}" ;
        madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                madsrdf:elementValue "{request.label}" ] 
                { '[ a madsrdf:DateNameElement ; madsrdf:elementValue "{}" ]'.format(request.date) if request.date else '' } ) ;
        madsrdf:fullerName [ a madsrdf:PersonalName ;
            rdfs:label "{request.fullerName}" ] ;
        { closeExternal.format(', '.join([ f'<{i}>' for i in request.hasCloseExternalAuthority])) if request.hasCloseExternalAuthority else '' }
        { exactExternal.format(', '.join([ f'<{i}>' for i in request.hasExactExternalAuthority])) if request.hasExactExternalAuthority else '' } 
        { hasVariant.format(', '.join(variants)) if request.hasVariant else '' } 
        { f'madsrdf:birthDate "{request.birthDate}" ;' if request.birthDate else '' } 
        { birthPlace.format(request.birthPlace) if request.birthPlace else '' } 
        { f'madsrdf:deathDate "{request.deathDate}" ;' if request.deathDate else '' } 
}} }}
"""
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(template)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [9]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(template)
response.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Line 21, column 13: Unresolved prefixed name: rdfs:label\n'

In [2]:
g = Graph()
g.parse("http://id.loc.gov/authorities/names/n80002329.madsrdf.rdf")
g.serialize('machado.nt')

<Graph identifier=Na0e645c62e64409daecc9f5efa8537a0 (<class 'rdflib.graph.Graph'>)>

In [5]:
g = Graph()
r = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/topic/sh85017405')
g.parse(r.content)
g.serialize("subject.nt")

<Graph identifier=N54510621a4ab41878d82b0362c98c28b (<class 'rdflib.graph.Graph'>)>